In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from sklearn.linear_model import LinearRegression, LogisticRegression
import duckdb


In [4]:
attributes_df = pd.read_csv("../info2950-final-project/Attribute_Description.csv")
medication_df = pd.read_csv("Medication_Data_Kaggle_V1.csv")
socio_eco_df = pd.read_csv("Socio_Economics_Data_Kaggle_V1.csv")
patient_data_training_df = pd.read_csv("Patient_Data_Training_Kaggle_V1.csv")

# Research Questions


# Data collection and cleaning

# Data description

# Data limitations

# Exploratory data analysis

# Questions for reviewers